In [1]:
import torch
from torch_geometric.data import Data

# Basics:

In [ ]:
""" graph connectivity in edge_index
first row: source nodes
second row: target nodes
"""
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
# X is a matrix of node features
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
# This is how a graph is represented in PyTorch Geometric
data = Data(x=x, edge_index=edge_index)


In [ ]:
"""
Checks if generated graph is valid
"""
data.validate(raise_on_error=True)

True

In [ ]:
# Data atributes
print(data.keys())

['x', 'edge_index']


In [6]:
# Shows node features
print(data['x'])

tensor([[-1.],
        [ 0.],
        [ 1.]])


In [8]:
# Shows number of nodes
print(data.num_nodes)

3


In [9]:
# Shows number of edges
print(data.num_edges)

4


In [ ]:
# Shows number of node features per node
print(data.num_node_features)

1


In [11]:
# Checks if there are isolated nodes
data.has_isolated_nodes()

False

In [12]:
# Checks for cycles
data.has_self_loops()

False

In [13]:
# Checks if is directed
data.is_directed()

False

In [14]:
# Transfer data object to GPU.
device = torch.device('cuda')
data = data.to(device)

## Datasets

In [67]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)



600
6
3


In [35]:
# Accesing the first graph in the dataset
data = dataset[4]
print(data.num_nodes)
print(data.num_edges)
print(data.num_node_features)
# Graph label
print(data["y"])

23
90
3
tensor([5])


In [37]:
"""
Dataset construction
"""
train_dataset = dataset[:540]
test_dataset = dataset[540:]
# Shuffling the dataset
dataset = dataset.shuffle()


## Dataloaders

In [ ]:
from torch_geometric.loader import DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)
for batch in loader:
    pass
print(batch)
print(batch.num_graphs)
print(batch[0])
"""
Batch by itself is a Data object, but it contains additional attributes
like batch.batch, that contains the mapping of nodes to graphs.
"""
print(batch.batch)

DataBatch(edge_index=[2, 2850], x=[716, 3], y=[24], batch=[716], ptr=[25])
24
Data(edge_index=[2, 100], x=[24, 3], y=[1])
tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5

In [56]:
"""The bach object stacks info from all graphs in the batch"""
print(batch["edge_index"].shape)
print(batch["x"].shape)

torch.Size([2, 2850])
torch.Size([716, 3])


## Transforms

In [71]:
import torch_geometric.transforms as T
"""Load new graph and turns it into a knn graph"""
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES',pre_transform=T.KNNGraph(k=6))

In [73]:
# We can also apply augmentation to the dataset
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES',pre_transform=T.KNNGraph(k=6), transform=T.RandomJitter(0.1))

/home/lu/Desktop/Trabajo/graph_scenarios/venv/lib/python3.11/site-packages/torch_geometric/data/dataset.py:240: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, pass `force_reload=True` explicitly to reload the dataset.
  warnings.warn(


## Learning Methods:

In [93]:
# Dataset Loading:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [104]:
print(dataset[0].num_nodes, dataset[0].num_edges, dataset[0].num_node_features)

2708 10556 1433


In [ ]:
# Very simple GCN model
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        # Defining the first convolutional layer: input features, output features
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        # Defining the  convolutional layer: input features, output features
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        # Retrieving node features and edge index
        x, edge_index = data.x, data.edge_index
        # Applying first convolutional layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Applying second convolutional layer
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [96]:
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in tqdm(range(10)):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 273.66it/s]

torch.Size([2708, 1433]) torch.Size([2, 10556])
torch.Size([2708, 16])
torch.Size([2708, 7])
torch.Size([2708, 1433]) torch.Size([2, 10556])
torch.Size([2708, 16])
torch.Size([2708, 7])
torch.Size([2708, 1433]) torch.Size([2, 10556])
torch.Size([2708, 16])
torch.Size([2708, 7])
torch.Size([2708, 1433]) torch.Size([2, 10556])
torch.Size([2708, 16])
torch.Size([2708, 7])
torch.Size([2708, 1433]) torch.Size([2, 10556])
torch.Size([2708, 16])
torch.Size([2708, 7])
torch.Size([2708, 1433]) torch.Size([2, 10556])
torch.Size([2708, 16])
torch.Size([2708, 7])
torch.Size([2708, 1433]) torch.Size([2, 10556])
torch.Size([2708, 16])
torch.Size([2708, 7])
torch.Size([2708, 1433]) torch.Size([2, 10556])
torch.Size([2708, 16])
torch.Size([2708, 7])
torch.Size([2708, 1433]) torch.Size([2, 10556])
torch.Size([2708, 16])
torch.Size([2708, 7])
torch.Size([2708, 1433]) torch.Size([2, 10556])
torch.Size([2708, 16])
torch.Size([2708, 7])
